In [24]:
import pandas as pd
from collections import Counter
from langdetect import detect
import langdetect
import numpy as np
import importlib
import os
current_dir = os.getcwd()
%cd ..
import textmining.text_miner 
import textmining.loader  as loader
import textmining.topic_modeler as tm
import textmining.significance_testing as significance_testing
import textmining.utility as utility
import textmining.analysis as analysis
import textmining.cluster_analysis as cluster_analysis
import textmining.machine_learning as machine_learning
import textmining.news_editorials_experiments as experiment
import textmining.cross_models_significance as cross

importlib.reload(textmining.machine_learning)
importlib.reload(textmining.news_editorials_experiments)
importlib.reload(textmining.cross_models_significance)
importlib.reload(textmining.cluster_analysis)

importlib.reload(textmining.topic_modeler)
importlib.reload(textmining.analysis)
importlib.reload(textmining.loader)
os.chdir(current_dir)

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

C:\Users\elba_ro\Documents\projects\nlpcss2020-editorials-analysis


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elba_ro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elba_ro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
trait = 'agreeableness'

In [7]:
load = loader.loader()

"corpus" is set. It contains the 6000 annotation


In [8]:
personality_df = load.load_personality_traits()
personality_df.set_index('id', inplace=True)


In [9]:
# Traints
def set_avg_to_low(row):
    row[trait] = 'LOW' if row[trait] == 'AVERAGE' else row[trait]
    return row
personality_df = personality_df.apply(set_avg_to_low, axis=1)
personality_df[[trait]]
personality_df[trait].value_counts()

HIGH    14
LOW     10
Name: agreeableness, dtype: int64

In [10]:
corpus = load.corpus
corpus = load.add_personality_label(personality_df[[trait]], trait)


"corpus" is set with personality: "a" , "b", ...


In [11]:
corpus[trait].value_counts()

HIGH    3500
LOW     2500
Name: agreeableness, dtype: int64

In [12]:
pd.crosstab(corpus[trait], corpus.effect_abstracted), print(''),print(''), pd.crosstab(corpus[trait], corpus.effect)

(effect_abstracted    1     2     3
 agreeableness                     
 HIGH               380  1050  2070
 LOW                307   940  1253,
 None,
 None,
 effect          1    2     3     4    5
 agreeableness                          
 HIGH           71  309  1050  1356  714
 LOW            72  235   940   844  409)

In [13]:
agreeable_maj_df = load.get_article_dfs_per_ideology(ideology = trait, include_content = True)

articles dataframe for ideology HIGH was created
The id of the df is the article id without txt
total con: 979
rounded TRAINGING data:  783
rounded Validation data:  0
rounded Test data:  196
total con: 1000
rounded TRAINGING data:  800
rounded Validation data:  0
rounded Test data:  200
"data_division" is set as dict with keys  dict_keys(['train', 'test'])
"corpus" is set with split_label: "train" "test"
"data_division" is set as dict with keys  dict_keys(['train', 'test'])
length of self.df:  979
length of self.data_division_df:  979
length of self.df:  979
articles dataframe for ideology LOW was created
The id of the df is the article id without txt
total con: 979
rounded TRAINGING data:  783
rounded Validation data:  0
rounded Test data:  196
total con: 1000
rounded TRAINGING data:  800
rounded Validation data:  0
rounded Test data:  200
"data_division" is set as dict with keys  dict_keys(['train', 'test'])
"corpus" is set with split_label: "train" "test"
"data_division" is set as 

In [17]:
high_df = agreeable_maj_df['HIGH']
#average_df = agreeable_maj_df['AVERAGE']
low_df = agreeable_maj_df['LOW']

In [20]:
pd.crosstab(low_df.majority, low_df.split_label)

split_label,test,train
majority,,
challenging,30,123
no_effect,76,291
reinforcing,90,369


In [21]:
pd.crosstab(high_df.majority, high_df.split_label)

split_label,test,train
majority,,
challenging,17,86
no_effect,43,162
reinforcing,136,535


In [ ]:
def check_if_all_0(df):
    return  df[(df['challenging']== 0) & (df['no_effect']== 0) & (df['reinforcing']== 0)]

check_if_all_0(high_df)
check_if_all_0(low_df)

In [22]:
print(high_df.majority.value_counts())

print()

#print(average_df.majority.value_counts())
#print()

print(low_df.majority.value_counts())


reinforcing    671
no_effect      205
challenging    103
Name: majority, dtype: int64

reinforcing    459
no_effect      367
challenging    153
Name: majority, dtype: int64


In [23]:
print(high_df.split_label.value_counts())
print()
#print(average_df.split_label.value_counts())
#print()
print(low_df.split_label.value_counts())
print()
print()
print(pd.crosstab(high_df.majority, high_df.split_label))
print()
#print(pd.crosstab(average_df.majority, average_df.split_label))
#print()
print(pd.crosstab(low_df.majority, low_df.split_label))

train    783
test     196
Name: split_label, dtype: int64

train    783
test     196
Name: split_label, dtype: int64


split_label  test  train
majority                
challenging    17     86
no_effect      43    162
reinforcing   136    535

split_label  test  train
majority                
challenging    30    123
no_effect      76    291
reinforcing    90    369


In [ ]:
high_df.rename(columns = {'majority': trait+'_high_majority'}, inplace = True)
#average_df.rename(columns = {'majority': trait+'_average_majority'}, inplace = True)
low_df.rename(columns = {'majority': trait+'_low_majority'}, inplace = True)

In [ ]:
data = pd.read_json('../data/articles_with_adu_liwc_lexicons_content.json', orient='records')
data.set_index('idx', inplace=True)
print(len(data))
data = data.merge(high_df[[trait+'_high_majority']], how="inner",left_index=True, right_index=True )
#data = data.merge(average_df[[trait+'_average_majority']], how="inner",left_index=True, right_index=True )
data = data.merge(low_df[[trait+'_low_majority']], how="inner",left_index=True, right_index=True )

data.drop(['liberal_majority', 'conservative_majority'], axis=1, inplace=True)
data.to_csv('../data/'+trait+'_HAL_all_features.csv')

# Training models

In [26]:
df = pd.read_csv('../data/'+trait+'_HAL_all_features.csv', index_col='idx')

In [27]:
training_df, test_df = analysis.get_train_test(df)

In [28]:

results_sqrt = experiment.run_experiments(df, ideologies=[trait+'_high_majority', 
                                                              #trait+'_average_majority',
                                                             trait+'_low_majority',],
                                              filename="../out/style_content_models_results/"+trait+"_HAL_experiments_sqrt",
                                              remove_outliers=True,
                                              normalize="sqrt")

running experiments for ideologies:  ['agreeableness_high_majority', 'agreeableness_low_majority'] 
 remove_outliers:  True 
 normalize:  sqrt
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
('liwc',)
performing gridsearch


2020-09-18 07:15:19,794 : ERROR : Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\programdata\miniconda3\lib\site-packages\joblib\parallel.py", line 909, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "c:\programdata\miniconda3\lib\site-packages\joblib\_parallel_backends.py", line 562, in wrap_future_result
    return future.result(timeout=timeout)
  File "c:\programdata\miniconda3\lib\concurrent\futures\_base.py", line 427, in result
    self._condition.wait(timeout)
  File "c:\programdata\miniconda3\lib\threading.py", line 296, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-1947f6312144>", line 6, in <module>
    normalize="sqrt")
  File "C:\Users\elba_ro\Documents\projects\nlpcss2020-editorials-analysi

TypeError: can only concatenate str (not "list") to str

In [ ]:
experiment.train_baseline(df, trait+'_high_majority')

In [ ]:
#experiment.train_baseline(df, trait+'_average_majority')

In [ ]:
experiment.train_baseline(df, trait+'_low_majority')

In [ ]:
for l in ['low', 'high']:
    print(l)
    print(results_minmax[ '{}_{}_majority'.format(trait, l)].macro.max() )
    print(results_log[ '{}_{}_majority'.format(trait, l)].macro.max() )
    print(results_sqrt[ '{}_{}_majority'.format(trait, l)].macro.max() )
    print(results_standard[ '{}_{}_majority'.format(trait, l)].macro.max() )


In [29]:
level= 'high'
PATH_RESULT= '../out/style_content_models_results/{}_HAL_experiments_{}_{}_{}_majority.csv'

high_cross_models_sqrt_df    = cross.run_model_pairs_significance(PATH_RESULT.format(trait,'sqrt',trait, level), 
                                                                  df, '{}_{}_majority'.format(trait, level), 
                                                                  'sqrt'    , 'sqrt_{}_{}'.format(trait, level) 
                                                                 )



preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('lemma',)) 2
('dummy',)
('lemma',)
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('liwc', 'nrc', 'mpqa_arg')) 2
('dummy',)
('liwc', 'nrc', 'mpqa_arg')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('nrc', 'adu')) 2
('dummy',)
('nrc', 'adu')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


...

c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('liwc', 'nrc', 'mpqa_arg'), ('mpqa_arg', 'mpqa_subjobg', 'adu', 'lemma')) 2
('liwc', 'nrc', 'mpqa_arg')
('mpqa_arg', 'mpqa_subjobg', 'adu', 'lemma')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('nrc', 'adu'), ('mpqa_arg', 'mpqa_subjobg', 'adu', 'lemma')) 2
('nrc', 'adu')
('mpqa_arg', 'mpqa_subjobg', 'adu', 'lemma')


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [30]:
high_cross_models_sqrt_df[high_cross_models_sqrt_df]

,model_pair,is_normal,stat,p_val,wilk_stat,wilk_p_val,significant,data
0,"((dummy,), (lemma,))",True,-0.232330,0.827684,6.0,0.685830,False,"{'('dummy',)': [0.27, 0.31, 0.36, 0.27, 0.35],..."
1,"((dummy,), (liwc, nrc, mpqa_arg))",False,-0.442074,0.681286,5.0,0.500184,False,"{'('dummy',)': [0.27, 0.31, 0.36, 0.27, 0.35],..."
2,"((dummy,), (nrc, adu))",True,-0.183643,0.863227,6.0,0.685830,False,"{'('dummy',)': [0.27, 0.31, 0.36, 0.27, 0.35],..."
3,"((dummy,), (mpqa_arg, mpqa_subjobg, adu, lemma))",True,-1.517539,0.203735,2.0,0.138011,False,"{'('dummy',)': [0.27, 0.31, 0.36, 0.27, 0.35],..."
4,"((lemma,), (liwc, nrc, mpqa_arg))",True,0.094551,0.929218,7.0,0.892738,False,"{'('lemma',)': [0.17448405253283303, 0.2810957..."
5,"((lemma,), (nrc, adu))",True,0.094589,0.929190,7.0,0.892738,False,"{'('lemma',)': [0.17448405253283303, 0.2810957..."
6,"((lemma,), (mpqa_arg, mpqa_subjobg, adu, lemma))",True,-1.097920,0.333890,4.0,0.345231,False,"{'('lemma',)': [0.17448405253283303, 0.2810957..."
7,"((liwc, nrc, mpqa_arg), (nrc, adu))",True,0.024974,0.981272,7.0,0.892738,False,"{'('liwc', 'nrc', 'mpqa_arg')': [0.28707893413..."
8,"((liwc, nrc, mpqa_arg), (mpqa_arg, mpqa_subjob...",True,-1.072010,0.344087,4.0,0.345231,False,"{'('liwc', 'nrc', 'mpqa_arg')': [0.28707893413..."
9,"((nrc, adu), (mpqa_arg, mpqa_subjobg, adu, lem...",True,-1.130589,0.321425,4.0,0.345231,False,"{'('nrc', 'adu')': [0.28819875776397513, 0.372..."


In [31]:
level= 'low'
PATH_RESULT= '../out/style_content_models_results/{}_HAL_experiments_{}_{}_{}_majority.csv'

low_cross_models_sqrt_df    = cross.run_model_pairs_significance(PATH_RESULT.format(trait,'sqrt',trait, level), 
                                                                  df, '{}_{}_majority'.format(trait, level), 
                                                                  'sqrt'    , 'sqrt_{}_{}'.format(trait, level) 
                                                                 )



preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('lemma',)) 2
('dummy',)
('lemma',)
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('liwc', 'adu')) 2
('dummy',)
('liwc', 'adu')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('liwc', 'adu', 'lemma')) 2
('dummy',)
('liwc', 'adu', 'lemma')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


...

c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('lemma',), ('liwc', 'adu', 'lemma')) 2
('lemma',)
('liwc', 'adu', 'lemma')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
agreeableness_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('liwc', 'adu'), ('liwc', 'adu', 'lemma')) 2
('liwc', 'adu')
('liwc', 'adu', 'lemma')


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [32]:
low_cross_models_sqrt_df

,model_pair,is_normal,stat,p_val,wilk_stat,wilk_p_val,significant,data
0,"((dummy,), (lemma,))",True,-1.107782,0.330081,4.0,0.345231,False,"{'('dummy',)': [0.42, 0.29, 0.29, 0.35, 0.14],..."
1,"((dummy,), (liwc, adu))",True,-1.914850,0.128028,2.0,0.138011,False,"{'('dummy',)': [0.42, 0.29, 0.29, 0.35, 0.14],..."
2,"((dummy,), (liwc, adu, lemma))",True,-2.582713,0.061152,1.0,0.079616,False,"{'('dummy',)': [0.42, 0.29, 0.29, 0.35, 0.14],..."
3,"((lemma,), (liwc, adu))",True,-1.680818,0.168093,2.0,0.138011,False,"{'('lemma',)': [0.34722222222222215, 0.3429211..."
4,"((lemma,), (liwc, adu, lemma))",True,-4.604875,0.009994,0.0,0.043114,True,"{'('lemma',)': [0.34722222222222215, 0.3429211..."
5,"((liwc, adu), (liwc, adu, lemma))",True,-0.947811,0.396896,4.0,0.345231,False,"{'('liwc', 'adu')': [0.3753258390355165, 0.474..."
